In [81]:
import torch 
import torch.nn.functional as F
import matplotlib as plt
%matplotlib inline

# read all the words 
words = words = open('/Users/m/dev/makemore/names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [2]:
len(words)

32033

In [147]:
# build the vocabulary of chars and the mappings to/from ints
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(stoi)

In [83]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  for w in words:

    #print(w)
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      #print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1]) # 80%
Xdev, Ydev = build_dataset(words[n1:n2]) # 10%
Xte, Yte = build_dataset(words[n2:]) # 10%

torch.Size([182580, 3]) torch.Size([182580])
torch.Size([22767, 3]) torch.Size([22767])
torch.Size([22799, 3]) torch.Size([22799])


In [14]:
C = torch.randn((27,2)) #initial random init
C[5]

tensor([ 1.0089, -0.4192])

In [60]:
emb = C[X]
emb.shape

torch.Size([228146, 3, 2])

In [61]:
W1 = torch.randn((6,100))
b1 = torch.randn(100)

In [ ]:
h = torch.tanh(emb.view(32,6) @ W1 + b1)
h.shape

In [26]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [27]:
logits = h @ W2 + b2
logits.shape

torch.Size([32, 27])

In [30]:
counts = logits.exp()
counts.shape

torch.Size([32, 27])

In [32]:
prob = counts / counts.sum(1, keepdim=True)
prob.shape

torch.Size([32, 27])

In [37]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(20.0160)

In [ ]:
### getting it all tidied up for trainig and inference ### 

In [84]:
Xtr.shape, Ytr.shape

(torch.Size([182580, 3]), torch.Size([182580]))

In [148]:
n_embd = 10 # the dimensionality of the char embedding vectors
n_hidden = 200 # the number of neurons in the hidden MLP laer

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((vocab_size, n_embd), generator=g)
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g)
b1 = torch.randn(n_hidden, generator=g)
W2 = torch.randn((n_hidden, vocab_size), generator=g)
b2 = torch.randn(vocab_size, generator=g)
parameters = [C, W1, b1, W2, b2]

In [149]:
sum(p.nelement() for p in parameters)

11897

In [150]:
for p in parameters:
    p.requires_grad = True

In [151]:
# training
max_steps = 200000
batch_size = 32

for i in range(max_steps):
    # make minibatch
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    # forward pass
    emb = C[Xtr[ix]] # (32, 3, 2)
    emb_cat = emb.view(emb.shape[0], -1) # concatenate the vectors to fit our required shape
    h_pre_act = emb_cat @ W1 + b1 # hidden layer pre-activation
    h = torch.tanh(h_pre_act) # hidden layer after activation
    logits = h @ W2 + b2 # output layer
    loss = F.cross_entropy(logits, Ytr[ix]) # this is negative log mean error
    # print('loss', loss.item())
    
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    # update
    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad

print('loss', loss.item())

loss 2.4457080364227295


In [152]:
# evalutaion 
@torch.no_grad() # do not track gradients
def split_loss(split):
    x,y = {
        'train': (Xtr, Ytr),
        'val': (Xdev, Ydev),
        'test': (Xte, Yte),
    }[split]

    emb = C[x] # # (N, block_size, n_embd)
    embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
    h = torch.tanh(embcat @ W1 + b1) #(N, n_hidden)
    logits = h @ W2 + b2 # #(N, vocab_size)
    loss = F.cross_entropy(logits, Ydev)

split_loss('train')
split_loss('val')

tensor(2.1607, grad_fn=<NllLossBackward0>)

In [153]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # forward pass
      emb = C[torch.tensor([context])] # (1,block_size,d)
      h = torch.tanh(emb.view(1, -1) @ W1 + b1)
      logits = h @ W2 + b2
      probs = F.softmax(logits, dim=1)
      # sample from distribution
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      # break if we come across '.' token
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))

carmah.
ambril.
khirmin.
rehtleenanden.
jazhetraymerynco.
geni.
ner.
kiah.
maiir.
kaleigh.
ham.
jore.
quint.
sulin.
alian.
quinterridearynix.
kael.
drudbwed.
emi.
abette.
